In [2]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.pmesh import ParamMesh, MetaMesh, MetaScene
from meshmaker.seam import Seam
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.planargraph import planargraph
#from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, isnear, near, loopO
from meshmaker.geometry import batch, slide, isnear, near, loop_normal, bbox
from meshmaker.mgl import show, MainShader, EdgeShader, WireShader
from meshmaker.plt import *
from collections import defaultdict
import json
from functools import partial

show = partial(show, programs=[MainShader(), EdgeShader(), WireShader(color=vec3.U().yz())])

In [2]:
#box = Mesh.cube_mesh2().subdivide().subdivide().subdivide().smooth()
box = Mesh.cube_mesh2().subdivide().subdivide().subdivide()
box.smooth(inplace=True)
box.normals = box.vertex_normals(smooth=True)
box.uvs = box.vertex_uvs(seams=box.angle_seams(0))
show(box)

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7fb9341b1550>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7fb9341b17b8>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7fb9341b1d68>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


# twisted cylinder example

In [2]:
#box = Mesh.cube_mesh2()
box = Mesh.cylinder_mesh(h=4, n=8, closed=True, meta='generic_14')

def someconstraints(mesh):
    # make some constraints
    fN = mesh.face_normals()
    constraints = {}
    for x in (mesh.nearest(vec3.X(-1)), mesh.nearest(vec3.X(+1))):
        xN = vec3.sum([fN[f] for f in mesh.v2f[x]]).nrm()
        constraints[x] = mesh.vertices[x] + xN
    return constraints

def someconstraints2(mesh):
    # make some constraints
    constraints = {}
    q = quat.rotz(np.pi / 2)
    #q = quat.rotz(0)
    fN = mesh.face_normals()
    bot, top = [], []
    for f, face in mesh:
        N = fN[f]
        if isnear(N.z, 1):
            top.extend(face)
        elif isnear(N.z, -1):
            bot.extend(face)
    botrot = [mesh.vertices[v].cp() for v in bot]
    toprot = q.rot([mesh.vertices[v].cp() for v in top])
    for i, v in zip(bot, botrot):
        constraints[i] = v
    for i, v in zip(top, toprot):
        constraints[i] = v
    return constraints

box = box.subdivide().subdivide().subdivide()

box.uvs = box.vertex_uvs(seams=box.angle_seams(np.pi / 2))

constraints = someconstraints2(box)
uvs = box.uvs
box = box.deform(constraints)
box.uvs = uvs
#box.uvs = box.vertex_uvs(seams=box.angle_seams(np.pi / 2))
#box = box.subdivide().smooth().deform(constraints).subdivide().smooth()

show((box, Mesh.cube_mesh(0.1)))

/home/cogle/dev/mine/meshmaker/meshmaker/mesh.py:324: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  xyz, _, _, _ = np.linalg.lstsq(L, delta)


Building Shader 1:<meshmaker.mgl.MainShader object at 0x7fcc0a460ac8>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7fcc0a460d30>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7fcc0a439320>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


# twisted cube example 

In [10]:
box = Mesh.cube_mesh2()

box = box.subdivide()

controls = {i: v for i, v in enumerate(box.vertices)}
constraints = {}

#box = box.subdivide()
#box = box.subdivide()

#constraints[0].trn(vec3.U(-1))
#constraints[6] += vec3.U(+1)
#constraints[6].trn(vec3.U(+1))

for i, v in controls.items():
    if v.z >= 1:
        constraints[i] = v.cp()

#top = [v for i, v in control.items() if v.z >= 1]
#(vec3.X() + vec3.Z()).trnps(quat.av(np.pi / 2, vec3.Y()).rot(top))
top = list(constraints.values())
quat.rotz(np.pi / 3).rot(top)

box = box.deform(constraints)

#box = box.subdivide()
#box = box.subdivide()

show(box)

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f68926b8278>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f68790c9198>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f68790c9400>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


# bent rod example

In [14]:
rod = Mesh.cube_mesh2()

rod.edgeloopsplit(0, 4)

show(rod)

Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f68926b8278>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f68790c9198>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f68790c9400>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


# trash line 

In [96]:
class Deformer:
    
    def __init__(self, mesh, control=None):
        self.mesh = mesh
        self.control = control

        self.weights = np.zeros((len(self.mesh.vertices), len(self.control.vertices)))
        for i, u in enumerate(self.mesh.vertices):
            if u is not None:
                for j, v in enumerate(self.control.vertices):
                    if v is not None:
                        self.weights[i, j] = -np.log(1 + u.d(v))
                        #self.weights[i, j] = np.exp(-u.d(v))
                        #self.weights[i, j] = 1 / (u.d(v) + 1)
                #self.weights[i, :] = [1 if x == self.weights[i, :].max() else 0 for x in self.weights[i, :]]
                #print(self.weights[i, :])
                #self.weights[i, np.where((self.weights[i, :] - self.weights[i, :].max()) < -0.1)] = 0
                self.weights[i, :] /= self.weights[i, :].sum()

    def deform(self, updated_control):
        """Apply deformation to self.mesh"""
        deformation = defaultdict(lambda : vec3.O())
        
        print(len(self.control.vertices))
        print(len(updated_control.vertices))
        
        for i, u in enumerate(self.mesh.vertices):
            if u is not None:
                for j, v in enumerate(self.control.vertices):
                    dv = updated_control.vertices[j] - v
                    deformation[i].trn(dv * self.weights[i, j])
        for i, dv in deformation.items():
            self.mesh.vertices[i].trn(dv)

In [54]:
box = Mesh.cube_mesh()
vec3(1, 0.2, 0.1).sclps(box.vertices)

controller = box.cp()

box = box.subdivide()
box = box.subdivide()
box = box.subdivide()
box = box.subdivide()

original = box.cp()

def deform_controller(c):
    face = [c.vertices[v] for v in c.faces[3]]
    vec3(1, 3, 3).sclps(face)
    return c

deformer = Deformer(box, controller)
    
updated = deform_controller(controller.cp())


deformer.deform(updated)

#box.normals = box.vertex_normals(smooth=True)
box.uvs = box.vertex_uvs(seams=box.angle_seams())

wires = []
wires.extend([[updated.vertices[i], updated.vertices[j]] for i, j in updated.e2f])
wires.extend([[deformer.control.vertices[i], deformer.control.vertices[j]] for i, j in deformer.control.e2f])
box.wires = lambda tf: [tf.transform(wire) for wire in wires]

#box = MetaMesh(box, texture='generic_14')
#original = MetaMesh(original, texture='generic_16')
#show(MetaScene(TForm(metas=[box, original])))

show(box)

8
8
Building Shader 1:<meshmaker.mgl.MainShader object at 0x7f510fdc3ef0>
Building Shader 2:<meshmaker.mgl.EdgeShader object at 0x7f510fdc3eb8>
Building Shader 3:<meshmaker.mgl.WireShader object at 0x7f510fdcf6d8>
Loaded texture: generic_0 (../resources/textures/generics/generic_0.png)


In [7]:
def deform(mesh, lattice):
    """Deform mesh vertices based on lattice"""
    pass